In [298]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model

# Preparacion de datos

In [292]:
def get_postulaciones(size=None):
    postulaciones = pd.read_csv('data/FiubaHasta15Abril/fiuba_4_postulaciones.csv', nrows =size).drop_duplicates(subset=['idpostulante', 'idaviso'], keep='last')
    columns_rename = {'idaviso': 'id_aviso', 'idpostulante': 'id_postulante', 'fechapostulacion': 'fecha_postulacion'}
    postulaciones = postulaciones.rename(columns=columns_rename)
    postulaciones['fecha_postulacion']=pd.to_datetime(postulaciones['fecha_postulacion'])
    return postulaciones

def get_vistas(size=None):
    vistas1 = pd.read_csv('data/fiuba_3_vistas.csv', nrows =size).drop_duplicates(subset=['idpostulante', 'idAviso'], keep='last')
    vistas2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_3_vistas.csv', nrows =size).drop_duplicates(subset=['idpostulante', 'idAviso'], keep='last')
    vistas3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_3_vistas.csv', nrows =size).drop_duplicates(subset=['idpostulante', 'idAviso'], keep='last')
    vistas = pd.concat([vistas1,vistas2,vistas3]).drop_duplicates(subset=['idpostulante', 'idAviso'], keep='last').reset_index(drop=True)
    vistas = vistas.groupby(['idpostulante', 'idAviso'], as_index=False)['timestamp'].count()

    if size is not None:
        vistas = vistas.head(size)
    columns_rename = {'idAviso': 'id_aviso', 'idpostulante': 'id_postulante', 'timestamp': 'visualizaciones'}
    vistas = vistas.rename(columns=columns_rename)        
    return vistas

def get_avisos_detalle():
    avisos1 = pd.read_csv('data/fiuba_6_avisos_detalle.csv')
    avisos2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_6_avisos_detalle.csv')
    avisos3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    avisos4 = pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')        
    avisos_detalle = pd.concat([avisos1,avisos2,avisos3,avisos4]).drop_duplicates(subset=['idaviso'], keep='last').reset_index(drop=True)
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    to_tipo_trabajo_nro={'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas':3, 'Pasantia':4,
       'Temporario':5, 'Por Contrato':6, 'Fines de Semana':7, 'Primer empleo':8,
       'Voluntario':9}
    to_nombre_area_numero = pd.Series(avisos_detalle['nombre_area'].unique()).to_dict()
    to_nombre_area_numero  = {v: k for k, v in to_nombre_area_numero.items()}
    avisos_detalle['nivel_laboral_nro']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    avisos_detalle['tipo_de_trabajo_nro'] = avisos_detalle['tipo_de_trabajo'].map(to_tipo_trabajo_nro)
    avisos_detalle['nombre_area_nro'] = avisos_detalle['nombre_area'].map(to_nombre_area_numero)

    h = FeatureHasher(n_features = 10, non_negative=True, input_type='string', dtype='float32')
    x = h.transform(avisos_detalle['titulo'])
    avisos_detalle['titulo'] = list(x.toarray())

    titulos_como_lista = avisos_detalle.titulo.apply(pd.Series)
    avisos_detalle = pd.merge(avisos_detalle, titulos_como_lista, left_index = True, right_index = True)
    avisos_detalle = avisos_detalle.drop(['titulo'], axis=1)  

    return avisos_detalle

def get_year_of_birth(postulantes_genero_edad):
    return (pd.to_datetime
            (postulantes_genero_edad['fechanacimiento'], errors='coerce', format='%Y-%m-%d')
            .dt.year)

def get_age(yearOfBirth):
    return 2018 - yearOfBirth
    
def get_age_range(yearOfBirth):
    age = get_age(yearOfBirth)
    if(age<25): return 'Entre 18 y 24'
    if(age<30): return 'Entre 25 y 30'
    if(age<35): return 'Entre 30 y 35'
    if(age<40): return 'Entre 35 y 40'
    if(age<45): return 'Entre 40 y 45'
    if(age<50): return 'Entre 45 y 50'
    return 'Mayor de 50'

def get_order_for_age_range():
    return ['Entre 18 y 24', 'Entre 25 y 30','Entre 30 y 35','Entre 35 y 40','Entre 40 y 45','Entre 45 y 50', 'Mayor de 50']

def get_postulantes_nivel_educativo_para(path):
    postulantes_nivel_educativo = pd.read_csv(path)
    columns_rename = {'idpostulante': 'id_postulante', 'nombre': 'formacion_postulante', 'estado': 'estado_formacion_postulante'}
    postulantes_nivel_educativo=postulantes_nivel_educativo.rename(columns=columns_rename)
    formacion_to_number={'Secundario' : 10, 'Otro': 20, 'Terciario/Técnico' : 30, 'Universitario' : 40, 'Posgrado' : 50,
    'Master' : 50, 'Doctorado' : 50}
    postulantes_nivel_educativo['formacion_postulante_numero']=postulantes_nivel_educativo['formacion_postulante'].map(formacion_to_number);
    estado_to_number = {'En Curso': 4, 'Abandonado': 0, 'Graduado': 8}
    postulantes_nivel_educativo['estado_formacion_postulante_numero']=postulantes_nivel_educativo['estado_formacion_postulante'].map(estado_to_number)
    postulantes_nivel_educativo['nivel_educativo_postulante_numero'] = postulantes_nivel_educativo['formacion_postulante_numero'] + postulantes_nivel_educativo['estado_formacion_postulante_numero']
    postulantes_nivel_educativo['nivel_educativo_postulante_texto'] = postulantes_nivel_educativo['formacion_postulante'] + ' - ' + postulantes_nivel_educativo['estado_formacion_postulante']
    relevant_columns = ['id_postulante','nivel_educativo_postulante_texto', 'nivel_educativo_postulante_numero']
    postulantes_nivel_educativo = postulantes_nivel_educativo[relevant_columns]
    grouped=postulantes_nivel_educativo.groupby(['id_postulante']).agg({'nivel_educativo_postulante_numero':['max']}) 
    df=grouped.reset_index()
    df.columns = ['id_postulante', 'maximo_nivel_educativo_postulante']
    return df

def get_postulantes_nivel_educativo():
    postulantes1 = get_postulantes_nivel_educativo_para('data/fiuba_1_postulantes_educacion.csv')
    postulantes2 = get_postulantes_nivel_educativo_para('data/FiubaDesde15Abril/fiuba_1_postulantes_educacion.csv')
    postulantes3 = get_postulantes_nivel_educativo_para('data/FiubaHasta15Abril/fiuba_1_postulantes_educacion.csv')
    return pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['id_postulante'], keep='last').reset_index(drop=True)

def get_postulantes_genero_edad():
    postulantes1 = pd.read_csv('data/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes_genero_edad = pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['idpostulante'], keep='last').reset_index(drop=True)
    postulantes_genero_edad['año_nacimiento_postulante']=get_year_of_birth(postulantes_genero_edad)
    postulantes_genero_edad['edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age, na_action=None)
    postulantes_genero_edad['rango_edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age_range, na_action=None)
    columns_rename = {'idpostulante': 'id_postulante', 'fechanacimiento': 'fecha_nacimiento_postulante', 'sexo': 'genero_postulante'}
    postulantes_genero_edad = postulantes_genero_edad.rename(columns=columns_rename)
    postulantes_genero_edad = postulantes_genero_edad[['id_postulante', 'genero_postulante', 'fecha_nacimiento_postulante', 'edad_postulante', 'rango_edad_postulante']]
    postulantes_genero_edad['genero_postulante_nro'] = postulantes_genero_edad['genero_postulante'].map({'FEM': 0, 'MASC': 1, 'NO_DECLARA': 2})
    return postulantes_genero_edad

def get_postulantes_limpios():
    postulantes = pd.merge(get_postulantes_genero_edad(), get_postulantes_nivel_educativo(), on='id_postulante', how='outer')
    order_for_columns = ['id_postulante','edad_postulante', 'genero_postulante', 'genero_postulante_nro', 'maximo_nivel_educativo_postulante']
    return postulantes[order_for_columns]

def get_detalle_postulaciones(size=None):
    postulaciones = get_postulaciones(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_postulaciones = pd.merge(postulantes, postulaciones, on='id_postulante', how='inner') 
    detalle_postulaciones = pd.merge(detalle_postulaciones, avisos, on='id_aviso', how='inner')
    
    return detalle_postulaciones

def get_detalle_vistas(size=None):
    vistas = get_vistas(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_vistas = pd.merge(postulantes, vistas, on='id_postulante', how='inner') 
    detalle_vistas = pd.merge(detalle_vistas, avisos, on='id_aviso', how='inner')
    return detalle_vistas

def x_entrenamiento():
    return ['edad_postulante', 'genero_postulante_nro', 'maximo_nivel_educativo_postulante', 'nivel_laboral_nro', 'tipo_de_trabajo_nro', 'nombre_area_nro',0,1,3,4,5,6,7,8,9]

def x_categoricas():
    return ['genero_postulante', 'maximo_nivel_educativo_postulante', 'nivel_laboral', 'tipo_de_trabajo', 'nombre_area']

def x_humanas():
    return ['id_postulante', 'id_aviso', 'titulo', 'descripcion', 'denominacion_empresa'] + x_categoricas

def y_entrenamiento():
    return ['sepostulo']

def columnas_relevantes_test_data():
    return ['id','id_aviso','id_postulante'] + x_entrenamiento()

def get_test_data():
    tests = pd.read_csv('data/Test/test_final_100k.csv')
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    tests = tests.rename(columns=columns_rename)
    tests = pd.merge(tests, get_avisos_detalle(), on='id_aviso', how='left')
    tests = pd.merge(tests, get_postulantes_limpios(), on='id_postulante', how='left')
    return tests[columnas_relevantes_test_data()]

def get_default_null_values(df):
    return {'edad_postulante':int(df['edad_postulante'].mean()), 'genero_postulante_nro':int(0), 'maximo_nivel_educativo_postulante':int(df['maximo_nivel_educativo_postulante'].mean()), 'nivel_laboral_nro':int(0), 'tipo_de_trabajo_nro':int(0), 'nombre_area_nro':int(0)}


def get_test_data_clean():
    tests = get_test_data()
    tests = tests.fillna(value=get_default_null_values(tests))
    return tests

def get_datos_entrenamiento(size=None):
    postulaciones_aplicadas = get_detalle_postulaciones(size)
    columnas_relevantes = x_entrenamiento() + y_entrenamiento()
    postulaciones_aplicadas['sepostulo'] = True
    postulaciones_no_aplicadas = get_detalle_vistas(size)
    postulaciones_no_aplicadas['sepostulo'] = False

    return postulaciones_aplicadas.append(postulaciones_no_aplicadas).drop_duplicates(subset=['id_aviso', 'id_postulante'], keep='first')[columnas_relevantes].dropna()

def get_predictor(set_entrenamiento):
    # Prepare feature and dependent variable along with categorical encoding
    X=pd.get_dummies(set_entrenamiento.loc[:, x_entrenamiento()])
    y=set_entrenamiento.loc[:, 'sepostulo']
    clf = svm.SVC()
    return clf.fit(X, y)

def get_random_forest_predictor(set_entrenamiento):
    X = set_entrenamiento.loc[:, x_entrenamiento()]
    Y = set_entrenamiento.loc[:, 'sepostulo']
    clf = RandomForestClassifier(n_estimators=15)
    return clf.fit(X, Y)

def get_sgd_classifier(set_entrenamiento):
    X = set_entrenamiento.loc[:, x_entrenamiento()]
    Y = set_entrenamiento.loc[:, 'sepostulo']
    clf = linear_model.SGDClassifier()
    return clf.fit(X, Y)

def get_general_predictor():
    return get_random_forest_predictor(get_datos_entrenamiento(10000))

def get_postulantes_a_personalizar():
    tests = pd.read_csv('data/Test/test_final_100k.csv')
    return set(tests['idpostulante'])

def get_personalized_predictor(cantidad_mininima_observaciones):
    datos_entrenamiento = get_datos_entrenamiento()
    postulantes = get_postulantes_a_personalizar()
    return get_random_forest_predictor(get_datos_entrenamiento(10000))

#El ensamble
def get_predictor_machine_avisos(cantidad_mininima_observaciones):
    machine = { generalPredictor : get_general_predictor(), personalizedPredictor : get_personalized_predictor(cantidad_mininima_observaciones)} 
    return machine

def obtener_predicciones(predictor, set_test):
    return predictor.predict(set_test[x_entrenamiento()])

def get_score(dataset, clasificador):
    X_train, X_test, y_train, y_test = train_test_split(dataset.loc[:, x_entrenamiento()], dataset.loc[:, 'sepostulo'], test_size=0.33, random_state=42)
    clasificador = clasificador.fit(X_train, y_train)    
    return clasificador.score(X_test, y_test)
 

In [301]:
set_entrenamiento = get_datos_entrenamiento()

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [302]:
avisos_detalle = get_avisos_detalle()

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [268]:
avisos_detalle.head()

,id_aviso,idpais,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,nivel_laboral_nro,tipo_de_trabajo_nro,nombre_area_nro,0,1,2,3,4,5,6
0,1000872556,1,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL,2.0,0,0,6.0,4.0,6.0,1.0,10.0,3.0,1.0
1,9240880,1,Agente\r\noficial Selecciona:</span></strong><...,Capital Federal,Buenos Aires,NaN,Full-time,Jefe / Supervisor / Responsable,Comercial,Agencia Oficial Alejandro Arizaga,3.0,0,1,8.0,12.0,12.0,0.0,13.0,7.0,2.0
2,1110185164,1,<p>Centro Médico Accord se encuentra en la bús...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Administración,Unión Personal,1.0,0,2,3.0,2.0,2.0,1.0,0.0,2.0,1.0
3,1110513885,1,<p>Importante correo privado ubicado en SAN MI...,Gran Buenos Aires,NaN,NaN,Full-time,Junior,Distribución,Asoko Tempo SA,1.0,0,3,4.0,1.0,4.0,0.0,3.0,1.0,0.0
4,1111034024,1,<p> </p><p><strong>Comercializadora incorpora ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,JELS SRL,2.0,0,1,9.0,4.0,8.0,1.0,13.0,4.0,1.0


In [303]:
set_entrenamiento['sepostulo'].value_counts()

True     4320492
False    3520293
Name: sepostulo, dtype: int64

In [295]:
set_entrenamiento.head()

,edad_postulante,genero_postulante_nro,maximo_nivel_educativo_postulante,nivel_laboral_nro,tipo_de_trabajo_nro,nombre_area_nro,0,1,3,4,5,6,7,8,9,sepostulo
0,37.0,0.0,48.0,2.0,0,58,6.0,6.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,True
1,40.0,0.0,40.0,2.0,0,27,2.0,2.0,0.0,0.0,2.0,0.0,0.0,1.0,3.0,True
2,40.0,0.0,40.0,2.0,0,56,4.0,6.0,5.0,3.0,3.0,1.0,3.0,2.0,4.0,True
3,40.0,0.0,40.0,2.0,0,56,2.0,3.0,1.0,3.0,1.0,1.0,2.0,0.0,3.0,True
4,40.0,0.0,40.0,2.0,0,27,3.0,2.0,0.0,1.0,3.0,0.0,2.0,1.0,6.0,True


# Armado de los sets de test y de entrenamiento

In [304]:
from sklearn.model_selection import train_test_split

x = set_entrenamiento.columns.values.tolist()
x.remove('sepostulo')

X_train, X_test, y_train, y_test = train_test_split(set_entrenamiento.loc[:, x], set_entrenamiento.loc[:, 'sepostulo'], test_size=0.2, random_state=43)


# Entrenamiento

In [311]:
clf = RandomForestClassifier(n_estimators=5)

predictor =  clf.fit(X_train, y_train)


In [313]:
predictor.score(X_test, y_test)

0.7064554123088441

# Prueba y resultados para la competencia

In [310]:
def obtener_predicciones(predictor, set_test):
    x = set_entrenamiento.columns.values.tolist()
    x.remove('sepostulo')    
    return predictor.predict(set_test[x]).astype(int)

def guardar_resultados(fileName, predictor, set_test):
    result = obtener_predicciones(predictor, set_test)
    set_test['sepostulo'] = result
    set_test[['id','sepostulo']].set_index('id').to_csv(fileName)
    return

set_test =get_test_data_clean()


guardar_resultados('100K ultimo.csv', predictor, set_test)
set_test['sepostulo'].value_counts()

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


1    50534
0    49466
Name: sepostulo, dtype: int64